<a href="https://colab.research.google.com/github/ucfilho/marquesgabi_August_2020/blob/master/03_ANN_NEW_DATA_CLASSIFICA_ago_04_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import random
import numpy as np
import cv2
import zipfile
from random import randint
from PIL import Image
import re
from sklearn.model_selection import train_test_split
#import scikit-image
import skimage
import pandas as pd

In [2]:
# Import the 'transform' module from 'skimage'
from skimage import transform 

In [3]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

In [4]:
# 03_ANN_NEW_DATA... only the grains in 882 are used for training 
#                    the ANN and segmented images are used to 
#                    train no-grain

# 03_ANN_NEW_DATA... only the grains in 882 are used for training the ANN and segmented images are used to train no-grain

In [5]:
!git clone https://github.com/ucfilho/marquesgabi_Julho_2020_using
%cd marquesgabi_Julho_2020_using
Transfere='FotosTreino882_and_Segm.zip'
file_name = zipfile.ZipFile(Transfere, 'r')
file_name.extractall()

fatal: destination path 'marquesgabi_Julho_2020_using' already exists and is not an empty directory.
/content/marquesgabi_Julho_2020_using


In [6]:
# First step: get the segmented file (photos stored in csv file)

# First step: get the segmented file (photos stored in csv file)

In [7]:
labels = [] #name files

with zipfile.ZipFile(Transfere, "r") as f:
  for f in f.namelist():
    labels.append(f)

In [8]:
Num=len(labels)
df=pd.read_csv(labels[0])
for i in range(1,Num):
  df_new=pd.read_csv(labels[i])
  df_new = df_new[~df_new['Type'].isin(['G'])] # drop grain row which is not in 882
  frames = [df, df_new]
  df= pd.concat(frames, ignore_index=True)

y_valor=df['Type']

In [9]:
print(df)

     Unnamed: 0 Type          0  ...        782        783  Width
0             0    G    0.00000  ...    0.68631    0.68769    NaN
1             1    G    0.00000  ...    0.49262    0.50909    NaN
2             2    G    0.45321  ...    0.54919    0.55439    NaN
3             3    G    0.75367  ...    0.48524    0.61876    NaN
4             4    G    0.94521  ...    0.98108    0.95838    NaN
..          ...  ...        ...  ...        ...        ...    ...
976           7    B   84.33883  ...    7.14600    7.38484  142.0
977           8    I  104.73698  ...    2.05457    0.38150  163.0
978           9    I   75.70248  ...  107.38844  106.00001  154.0
979          10    I   56.64465  ...    3.15316    0.40107  187.0
980          11    B  111.50165  ...    0.00000    0.00000  194.0

[981 rows x 787 columns]


In [10]:
print(df.head())

   Unnamed: 0 Type        0        1  ...      781      782      783  Width
0           0    G  0.00000  0.00000  ...  0.68642  0.68631  0.68769    NaN
1           1    G  0.00000  0.00000  ...  0.47277  0.49262  0.50909    NaN
2           2    G  0.45321  0.52257  ...  0.54645  0.54919  0.55439    NaN
3           3    G  0.75367  0.64590  ...  0.43356  0.48524  0.61876    NaN
4           4    G  0.94521  0.89736  ...  0.98942  0.98108  0.95838    NaN

[5 rows x 787 columns]


In [11]:
print(df)

     Unnamed: 0 Type          0  ...        782        783  Width
0             0    G    0.00000  ...    0.68631    0.68769    NaN
1             1    G    0.00000  ...    0.49262    0.50909    NaN
2             2    G    0.45321  ...    0.54919    0.55439    NaN
3             3    G    0.75367  ...    0.48524    0.61876    NaN
4             4    G    0.94521  ...    0.98108    0.95838    NaN
..          ...  ...        ...  ...        ...        ...    ...
976           7    B   84.33883  ...    7.14600    7.38484  142.0
977           8    I  104.73698  ...    2.05457    0.38150  163.0
978           9    I   75.70248  ...  107.38844  106.00001  154.0
979          10    I   56.64465  ...    3.15316    0.40107  187.0
980          11    B  111.50165  ...    0.00000    0.00000  194.0

[981 rows x 787 columns]


In [12]:
df_teste=pd.read_csv(labels[2])
print(df_teste.head())

   Unnamed: 0  Width Type          0  ...       780       781        782        783
0           0    157    I   71.90012  ...  49.57118  35.95513    7.75954    1.03862
1           1    124    I  127.06763  ...  94.57544  98.71695  100.52861  100.84391
2           2    182    G   87.10651  ...   5.33136   5.84024    5.89941    5.65089
3           3    101    I   99.01176  ...  35.15087  41.39006   43.76875   44.09146
4           4    183    I   56.30844  ...  52.96656  53.65711   53.44656   53.11703

[5 rows x 787 columns]


In [13]:
print(df.tail())

     Unnamed: 0 Type          0  ...        782        783  Width
976           7    B   84.33883  ...    7.14600    7.38484  142.0
977           8    I  104.73698  ...    2.05457    0.38150  163.0
978           9    I   75.70248  ...  107.38844  106.00001  154.0
979          10    I   56.64465  ...    3.15316    0.40107  187.0
980          11    B  111.50165  ...    0.00000    0.00000  194.0

[5 rows x 787 columns]


In [14]:
Fotos=df.drop(['Unnamed: 0','Type','Width'], axis=1)

In [15]:
df=df.drop(['Unnamed: 0','Type','Width'], axis=1)

In [16]:
print(df.describe())

                0           1           2  ...         781         782         783
count  981.000000  981.000000  981.000000  ...  981.000000  981.000000  981.000000
mean     7.834253    7.847765    7.784539  ...    5.294938    5.215285    5.291887
std     23.507544   23.434137   23.360003  ...   18.945649   19.327633   19.863527
min      0.000000    0.000000    0.000000  ...    0.000000    0.000000    0.000000
25%      0.311510    0.309190    0.313200  ...    0.332510    0.336190    0.329620
50%      0.499150    0.490170    0.490990  ...    0.483030    0.479810    0.489680
75%      0.640190    0.644030    0.652580  ...    0.639650    0.640320    0.637210
max    127.067630  126.206040  120.224750  ...  108.437500  114.958580  121.952670

[8 rows x 784 columns]


In [17]:
print(np.array(Fotos).shape )

(981, 784)


# Second step: create the ann to evaluate which is a grain in photos segmented


In [18]:
images28=np.array(df)

In [19]:
#print(y_valor)

In [20]:
y_new=[]
Grain=0
Others=0
Cont=-1
Others_Local=[]
Grain_Local=[]
for x in y_valor:
  Cont=Cont+1
  if re.search('G', x):
    y_new.append(1)
    Grain=Grain+1
    Grain_Local.append(Cont)
  else:
    y_new.append(2)
    Others=Others+1
    Others_Local.append(Cont)

In [21]:
print('Graos=',Grain)
print('Others=',Others)
print('Others local',Others_Local)
print('Grain local',Grain_Local)

sampled_list = random.sample(Others_Local,Grain)

Graos= 245
Others= 736
Others local [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 26

In [22]:
print('Graos=',Grain)
print('Others=',Others)
print('Others local',Others_Local)
sampled_list = random.sample(Others_Local,Grain)


Graos= 245
Others= 736
Others local [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 26

In [23]:
print('Graos=',Grain)
print('Others=',Others)
print('Others local',Others_Local)
print('Grain local',Grain_Local)


Graos= 245
Others= 736
Others local [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 26

In [24]:
x_new=[]
y_rede=[]
for i in range(Grain):
  x_new.append(Fotos.iloc[Grain_Local[i]])
  x_new.append(Fotos.iloc[sampled_list[i]])
  y_rede.append(y_new[Grain_Local[i]])
  y_rede.append(y_new[sampled_list[i]])


In [25]:
np.array(x_new).shape

(490, 784)

In [26]:
#pd.DataFrame(y_rede).describe()

In [27]:
# y_valor=np.copy(y_new)
y_total=np.copy(y_rede)

In [28]:
#Define data train and data test
#images28=Fotos
images28=x_new #Fotos
W_train, W_test, yw_train, yw_test = train_test_split(np.array(images28), np.array(y_total), 
                                                    test_size=0.30, 
                                                    random_state=42)

In [29]:
print(len(y_total))
print(len(yw_test))
print(len(yw_train))
A=490*0.3
print(A)

490
147
343
147.0


In [30]:
train_images=W_train #imagens utilizadas para o treino
train_labels=yw_train # resposta esperada para o treino
test_images=W_test
test_labels=yw_test

In [31]:
#print(len(test_labels))

In [32]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10)
])

In [33]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [34]:
model.fit(train_images, train_labels, epochs=500)

Epoch 1/500
11/11 [==============================] - 0s 2ms/step - loss: 3.9025 - accuracy: 0.4927
Epoch 2/500
11/11 [==============================] - 0s 2ms/step - loss: 1.2084 - accuracy: 0.5015
Epoch 3/500
11/11 [==============================] - 0s 2ms/step - loss: 1.0645 - accuracy: 0.5102
Epoch 4/500
11/11 [==============================] - 0s 2ms/step - loss: 0.9181 - accuracy: 0.5219
Epoch 5/500
11/11 [==============================] - 0s 2ms/step - loss: 1.2946 - accuracy: 0.5102
Epoch 6/500
11/11 [==============================] - 0s 2ms/step - loss: 0.7846 - accuracy: 0.5918
Epoch 7/500
11/11 [==============================] - 0s 2ms/step - loss: 0.8570 - accuracy: 0.5306
Epoch 8/500
11/11 [==============================] - 0s 2ms/step - loss: 0.5959 - accuracy: 0.6997
Epoch 9/500
11/11 [==============================] - 0s 2ms/step - loss: 0.6506 - accuracy: 0.6268
Epoch 10/500
11/11 [==============================] - 0s 2ms/step - loss: 0.8015 - accuracy: 0.6968
Epoch 11/

In [35]:
#ANN das imagens
x=np.array(W_test)
logits = model(x, training=False)
prediction = tf.argmax(logits, axis=1, output_type=tf.int32)
#print(prediction)

In [36]:
print(x)

[[0.62516 0.67814 0.7257  ... 0.34312 0.32268 0.31357]
 [0.52206 0.51394 0.51079 ... 0.75293 0.64692 0.61091]
 [0.15494 0.1429  0.13764 ... 0.15702 0.08283 0.09974]
 ...
 [0.67142 0.67645 0.68294 ... 0.81235 0.7893  0.73144]
 [0.5801  0.59905 0.64345 ... 0.44491 0.43344 0.44076]
 [0.58283 0.59543 0.61642 ... 0.45384 0.2914  0.14095]]


In [37]:
y_valor=np.copy(yw_test)
data = {'y_Actual': y_valor,
        'y_Predicted': prediction
        }

df = pd.DataFrame(data, columns=['y_Actual','y_Predicted'])
#print (df)

In [38]:
print(len(y_valor))

147


In [39]:
confusion_matrix = pd.crosstab(df['y_Actual'], df['y_Predicted'], rownames=['Actual'], colnames=['Predicted'])
print(confusion_matrix)

Predicted   1   2
Actual           
1          65  12
2           8  62
